## End to end Deep Learning Project Using Simple RNN

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

sentence_length = np.max(np.array([len(s.split(' ')) for s in corpus])) + 3

In [3]:
# Load the imdb dataset

max_features=10000 ##vocabulary size
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=max_features)

# Print the shape of the data
print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}')

17464789/17464789 [==============================] - 1s 0us/step
Training data shape: (25000,), Training labels shape: (25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [7]:
print(X_train[0],y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1


### These are indexes of each word in one hot representation
* 1st word in one hot vector of 1's and 0's is 1 at index 1
* 2nd word in one hot vector of 1's and 0's is 1 at index 14

### y_train is just 1 -- positive sentiment or 0 -- negative sentiment

In [8]:
# Inspect a sample review and its label
sample_review=X_train[0]
sample_label=y_train[0]

print(f"Sample review (as integers):{sample_review}")
print(f'Sample label: {sample_label}')

Sample review (as integers):[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label: 1


In [14]:
# MApping of words index bacl to words(for understanding)
word_index=imdb.get_word_index()
print({k: word_index[k] for k in list(word_index)[:10]})
reverse_word_index = {value: key for key, value in word_index.items()}
{k: reverse_word_index[k] for k in list(reverse_word_index)[:10]}

{'fawn': 34701, 'tsukino': 52006, 'nunnery': 52007, 'sonja': 16816, 'vani': 63951, 'woods': 1408, 'spiders': 16115, 'hanging': 2345, 'woody': 2289, 'trawling': 52008}


{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling'}

In [15]:
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in sample_review])
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

### Padding

In [16]:
max_len=500
X_train_pad = sequence.pad_sequences(X_train,maxlen=max_len) # default: pre
X_test_pad = sequence.pad_sequences(X_test, maxlen=max_len)

In [17]:
## Train Simple RNN
model=Sequential()

# Embedding Layer: converts words to vector
# 128 is size of feature representaion vector for each word
model.add(Embedding(max_features, 128, input_length=max_len)) ## Embedding Layers

model.add(SimpleRNN(128, activation='relu'))

# 1 is beciz only 1 output
model.add(Dense(1,activation="sigmoid"))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
# Create an instance of EarlyStoppping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [21]:
## Train the model with early sstopping
history=model.fit(
    X_train_pad, y_train, 
    epochs=10, 
    batch_size=32,
    validation_split = 0.2,
    callbacks=[earlystopping]
)

Epoch 1/10
625/625 [==============================] - 111s 176ms/step - loss: 929.1405 - accuracy: 0.6145 - val_loss: 0.5522 - val_accuracy: 0.7168
Epoch 2/10
625/625 [==============================] - 112s 178ms/step - loss: 0.4512 - accuracy: 0.7850 - val_loss: 0.4165 - val_accuracy: 0.8154
Epoch 3/10
625/625 [==============================] - 111s 177ms/step - loss: 0.2884 - accuracy: 0.8779 - val_loss: 0.3999 - val_accuracy: 0.8348
Epoch 4/10
625/625 [==============================] - 110s 176ms/step - loss: 0.1997 - accuracy: 0.9218 - val_loss: 0.4222 - val_accuracy: 0.8188
Epoch 5/10
625/625 [==============================] - 110s 175ms/step - loss: 0.2577 - accuracy: 0.9017 - val_loss: 0.5042 - val_accuracy: 0.8066
Epoch 6/10
625/625 [==============================] - 110s 176ms/step - loss: 0.1453 - accuracy: 0.9464 - val_loss: 0.4685 - val_accuracy: 0.8248
Epoch 7/10
625/625 [==============================] - 111s 177ms/step - loss: 0.1078 - accuracy: 0.9618 - val_loss: 0.5414

In [22]:
# Save model file
model.save('simple_rnn_imdb.h5')